In [1]:
import time
from datetime import timedelta

import html
import re

import os
import ntpath

import subprocess as sp

import numpy as np
import pandas as pd
import xml.etree.ElementTree as et

import dask
import dask.dataframe as dd
import dask.bag as bd
from dask.distributed import Client
from dask_jobqueue import SLURMCluster

## Enter full path to xml file

In [2]:
xml_file = 'pmd1/pmdxmlreports_xml/pmd*.xml'

## Read xml file in block sizes

In [3]:
## Read xml file in block sizes
bg = bd.read_text(xml_file, blocksize='10MB', linedelimiter='</violation>\n')

In [4]:
bg

dask.bag<bag-from-delayed, npartitions=577>

In [5]:
bg.take(1)

('<?xml version="1.0" encoding="UTF-8"?>\n<pmd xmlns="http://pmd.sourceforge.net/report/2.0.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://pmd.sourceforge.net/report/2.0.0 http://pmd.sourceforge.net/report_2_0_0.xsd" version="6.38.0" timestamp="2022-02-08T22:01:47.072">\n<file name="/scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/pmd1/mvn_apps/my-javacodeanalysis-app0/src/main/java/myjavacodeanalysis/Code_1001005_1000723_311_0.java">\n<violation beginline="2" endline="2" begincolumn="1" endcolumn="21" rule="UnnecessaryImport" ruleset="Code Style" class="Code_1001005_1000723_311_0" externalInfoUrl="https://pmd.github.io/pmd-6.38.0/pmd_rules_java_codestyle.html#unnecessaryimport" priority="4">\nUnused import \'com.sun.jna.*\'\n</violation>\n',)

In [6]:
bg.take(5)

('<?xml version="1.0" encoding="UTF-8"?>\n<pmd xmlns="http://pmd.sourceforge.net/report/2.0.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://pmd.sourceforge.net/report/2.0.0 http://pmd.sourceforge.net/report_2_0_0.xsd" version="6.38.0" timestamp="2022-02-08T22:01:47.072">\n<file name="/scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/pmd1/mvn_apps/my-javacodeanalysis-app0/src/main/java/myjavacodeanalysis/Code_1001005_1000723_311_0.java">\n<violation beginline="2" endline="2" begincolumn="1" endcolumn="21" rule="UnnecessaryImport" ruleset="Code Style" class="Code_1001005_1000723_311_0" externalInfoUrl="https://pmd.github.io/pmd-6.38.0/pmd_rules_java_codestyle.html#unnecessaryimport" priority="4">\nUnused import \'com.sun.jna.*\'\n</violation>\n',
 '<violation beginline="6" endline="22" begincolumn="10" endcolumn="3" rule="UnnecessaryModifier" ruleset="Code Style" class="Code

## Replace the xml header with empty string '<xml ...>' => ''

In [7]:
## Replace the xml header with empty string '<xml ...>' => ''
bg = bg.map(lambda line: re.sub(r'\s*<\s*\?xml.*\?>\n', '', line)) 

In [8]:
bg.take(5)

('<pmd xmlns="http://pmd.sourceforge.net/report/2.0.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://pmd.sourceforge.net/report/2.0.0 http://pmd.sourceforge.net/report_2_0_0.xsd" version="6.38.0" timestamp="2022-02-08T22:01:47.072">\n<file name="/scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/pmd1/mvn_apps/my-javacodeanalysis-app0/src/main/java/myjavacodeanalysis/Code_1001005_1000723_311_0.java">\n<violation beginline="2" endline="2" begincolumn="1" endcolumn="21" rule="UnnecessaryImport" ruleset="Code Style" class="Code_1001005_1000723_311_0" externalInfoUrl="https://pmd.github.io/pmd-6.38.0/pmd_rules_java_codestyle.html#unnecessaryimport" priority="4">\nUnused import \'com.sun.jna.*\'\n</violation>\n',
 '<violation beginline="6" endline="22" begincolumn="10" endcolumn="3" rule="UnnecessaryModifier" ruleset="Code Style" class="Code_1001005_1000723_311_0$Code" externalInf

## Replace the pmd header with empty string '<pmd  ...>' => ''

In [9]:
## Replace the xml header with empty string '<xml ...>' => ''
bg = bg.map(lambda line: re.sub(r'\s*<\s*pmd.*>\n', '', line)) 

In [10]:
bg.take(5)

('<file name="/scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/pmd1/mvn_apps/my-javacodeanalysis-app0/src/main/java/myjavacodeanalysis/Code_1001005_1000723_311_0.java">\n<violation beginline="2" endline="2" begincolumn="1" endcolumn="21" rule="UnnecessaryImport" ruleset="Code Style" class="Code_1001005_1000723_311_0" externalInfoUrl="https://pmd.github.io/pmd-6.38.0/pmd_rules_java_codestyle.html#unnecessaryimport" priority="4">\nUnused import \'com.sun.jna.*\'\n</violation>\n',
 '<violation beginline="6" endline="22" begincolumn="10" endcolumn="3" rule="UnnecessaryModifier" ruleset="Code Style" class="Code_1001005_1000723_311_0$Code" externalInfoUrl="https://pmd.github.io/pmd-6.38.0/pmd_rules_java_codestyle.html#unnecessarymodifier" priority="3">\nUnnecessary modifier \'static\' on interface \'Code\': member interfaces are implicitly static\n</violation>\n',
 '<violation beginline="7" endline="8" begincolum

### Drop wherever the Errors tage occurs

In [11]:
# replace all the '<error>' with '</error>' so that we can access it using xml.etree.ElementTree
#bg = bg.map(lambda line: re.sub(r'\s*<\s*error.*>\n|.*/error>\n', '', line))
# drop the content within the error tags <![CDATA[...]]>
#bg = bg.map(lambda line: re.sub(r'<\!.*', '', line))
# drop both error and content within the error tags <error ...><![CDATA[...]]></error>
bg = bg.map(lambda line: re.sub(r'\s*<\s*error.*>\n|<\!.*|.*/error>\n', '', line))

In [12]:
bg.take(5)

('<file name="/scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/pmd1/mvn_apps/my-javacodeanalysis-app0/src/main/java/myjavacodeanalysis/Code_1001005_1000723_311_0.java">\n<violation beginline="2" endline="2" begincolumn="1" endcolumn="21" rule="UnnecessaryImport" ruleset="Code Style" class="Code_1001005_1000723_311_0" externalInfoUrl="https://pmd.github.io/pmd-6.38.0/pmd_rules_java_codestyle.html#unnecessaryimport" priority="4">\nUnused import \'com.sun.jna.*\'\n</violation>\n',
 '<violation beginline="6" endline="22" begincolumn="10" endcolumn="3" rule="UnnecessaryModifier" ruleset="Code Style" class="Code_1001005_1000723_311_0$Code" externalInfoUrl="https://pmd.github.io/pmd-6.38.0/pmd_rules_java_codestyle.html#unnecessarymodifier" priority="3">\nUnnecessary modifier \'static\' on interface \'Code\': member interfaces are implicitly static\n</violation>\n',
 '<violation beginline="7" endline="8" begincolum

In [13]:
# test if the error tags are dropped
bg.filter(lambda line: line.find('<error') >= 0).take(5)

()

In [14]:
# test if the content drop the content within the error tags are dropped
bg.filter(lambda line: line.find('<![CDATA') >= 0).take(5)

()

### Drop wherever the file tage occurs
- Since we are intrested in getting the 'violations'
  

In [15]:
# replace all the '<file>' with '</file>' so that we can access it using xml.etree.ElementTree
#bg = bg.map(lambda line: re.sub(r'\s*<\s*file.*>|.*/file>', '', line))
bg = bg.map(lambda line: re.sub(r'\s*<\s*file.*>\n|.*/file>\n', '', line))

In [16]:
bg.take(5)

('<violation beginline="2" endline="2" begincolumn="1" endcolumn="21" rule="UnnecessaryImport" ruleset="Code Style" class="Code_1001005_1000723_311_0" externalInfoUrl="https://pmd.github.io/pmd-6.38.0/pmd_rules_java_codestyle.html#unnecessaryimport" priority="4">\nUnused import \'com.sun.jna.*\'\n</violation>\n',
 '<violation beginline="6" endline="22" begincolumn="10" endcolumn="3" rule="UnnecessaryModifier" ruleset="Code Style" class="Code_1001005_1000723_311_0$Code" externalInfoUrl="https://pmd.github.io/pmd-6.38.0/pmd_rules_java_codestyle.html#unnecessarymodifier" priority="3">\nUnnecessary modifier \'static\' on interface \'Code\': member interfaces are implicitly static\n</violation>\n',
 '<violation beginline="7" endline="8" begincolumn="19" endcolumn="49" rule="UnnecessaryModifier" ruleset="Code Style" class="Code_1001005_1000723_311_0$Code" variable="VariableDeclaratorId" externalInfoUrl="https://pmd.github.io/pmd-6.38.0/pmd_rules_java_codestyle.html#unnecessarymodifier" prior

### Filter out just the violations

In [17]:
## Filter out just the rows
bg = bg.filter(lambda line: line.find('<violation') >= 0)
#bg = bg.filter(lambda line:  re.sub(r'\s*<\s*violation.*>\n|.*/violation>\n', '', line) >= 0)
#bg.filter(lambda line: line.find('<violation' or 'violation>') >= 0).take(5)
#bg.filter(lambda line:  re.findall(r'\s*<\s*violation.*>\n|.*/violation>\n', line)).take(5)
#bg = bg.filter(lambda line:  re.findall(r'\s*<\s*violation.*>\n.*|.*\n|<.*/violation>\n', line))

In [18]:
bg.take(5)

('<violation beginline="2" endline="2" begincolumn="1" endcolumn="21" rule="UnnecessaryImport" ruleset="Code Style" class="Code_1001005_1000723_311_0" externalInfoUrl="https://pmd.github.io/pmd-6.38.0/pmd_rules_java_codestyle.html#unnecessaryimport" priority="4">\nUnused import \'com.sun.jna.*\'\n</violation>\n',
 '<violation beginline="6" endline="22" begincolumn="10" endcolumn="3" rule="UnnecessaryModifier" ruleset="Code Style" class="Code_1001005_1000723_311_0$Code" externalInfoUrl="https://pmd.github.io/pmd-6.38.0/pmd_rules_java_codestyle.html#unnecessarymodifier" priority="3">\nUnnecessary modifier \'static\' on interface \'Code\': member interfaces are implicitly static\n</violation>\n',
 '<violation beginline="7" endline="8" begincolumn="19" endcolumn="49" rule="UnnecessaryModifier" ruleset="Code Style" class="Code_1001005_1000723_311_0$Code" variable="VariableDeclaratorId" externalInfoUrl="https://pmd.github.io/pmd-6.38.0/pmd_rules_java_codestyle.html#unnecessarymodifier" prior

### Use ElementTree to get all the attrbutes and text in xmlor html tags

In [19]:
# This will give us just the attributes in the violation tag: <violation ...>
#violations_rows_bd = violations_rows_bd.map(lambda row: et.fromstring(row).attrib)

In [20]:
# This will give us just the text enclosed in the violation tag: <violation ...>
#violations_rows_bd = violations_rows_bd.map(lambda row: {'text':et.fromstring(row).text})

- To get both the the attrbutes and text in the violation tag 
    - We use add_text_to_dict_attrb() to achieve that

In [21]:
# Add extra key value to dict if they do not exixt
def add_key_val_if_not_exist(_dict):
    _dict
    keys = [
        'beginline',
        'endline',
        'begincolumn',
        'endcolumn', 
        'rule',
        'ruleset',
        'class',
        'method',
        'externalInfoUrl',
        'priority',
        'msg'
    ]
    value=''
    for key in keys:
        if key not in _dict:
            _dict[key] = value
    return _dict

In [22]:
# Convert text in tag to key/value
def update_dict_key_wt_text_as_value(tag, value, _dict):
    if tag == 'violation': 
        _dict['msg'] = value
    else:
        _dict = add_key_val_if_not_exist({})
    return _dict

In [23]:
#https://stackoverflow.com/questions/43921237/access-nested-children-in-xml-file-parsed-with-elementtree
def content(root):
    _dict = {}
    #list_of_dicts = []
    try:
        for child in root.iter():
            # update dict with text as values
            _dict = update_dict_key_wt_text_as_value(child.tag, child.text, child.attrib)
            # update dict with keys that do not exist
            _dict = add_key_val_if_not_exist(_dict)
            #list_of_dicts.append(_dict)
    except ParseError:
        # pass an empty dict eo update all keys with empty values
        #list_of_dicts.append(add_key_val_if_not_exist({}))
        _dict = add_key_val_if_not_exist({})

    #return list_of_dicts
    return _dict

In [24]:
## Use ElementTree to get all the attrbutes in xml tags
#bg = bg.map(lambda row: et.fromstring(row).attrib)
bg = bg.map(lambda row: content(et.fromstring(row)))

In [25]:
bg.take(5)

({'beginline': '2',
  'endline': '2',
  'begincolumn': '1',
  'endcolumn': '21',
  'rule': 'UnnecessaryImport',
  'ruleset': 'Code Style',
  'class': 'Code_1001005_1000723_311_0',
  'externalInfoUrl': 'https://pmd.github.io/pmd-6.38.0/pmd_rules_java_codestyle.html#unnecessaryimport',
  'priority': '4',
  'msg': "\nUnused import 'com.sun.jna.*'\n",
  'method': ''},
 {'beginline': '6',
  'endline': '22',
  'begincolumn': '10',
  'endcolumn': '3',
  'rule': 'UnnecessaryModifier',
  'ruleset': 'Code Style',
  'class': 'Code_1001005_1000723_311_0$Code',
  'externalInfoUrl': 'https://pmd.github.io/pmd-6.38.0/pmd_rules_java_codestyle.html#unnecessarymodifier',
  'priority': '3',
  'msg': "\nUnnecessary modifier 'static' on interface 'Code': member interfaces are implicitly static\n",
  'method': ''},
 {'beginline': '7',
  'endline': '8',
  'begincolumn': '19',
  'endcolumn': '49',
  'rule': 'UnnecessaryModifier',
  'ruleset': 'Code Style',
  'class': 'Code_1001005_1000723_311_0$Code',
  'vari

## Start a Dask cluster using SLURM jobs as workers.

There are a couple of things we need to configure here:

disabling the mechanism to write on disk when workers run out of memory,
memory, CPUs, maximum time and number of workers per SLURM job,
dask folders for log files and workers data.
We recommend putting the log folder and workers data folders in your /nesi/nobackup/<project_code> folder, most indicated for temporary files (see NeSI File Systems and Quotas).

All of these options can be set in configuration files, see Dask configuration and Dask jobqueue configuration for more information.

In [26]:
#http://jobqueue.dask.org/en/latest/generated/dask_jobqueue.SLURMCluster.html
dask.config.set(
    {
        "distributed.worker.memory.target": False,  # avoid spilling to disk
        "distributed.worker.memory.spill": False,  # avoid spilling to disk
    }
)
cluster = SLURMCluster(
    cores=10, #cores=24, # we set each job to have 1 Worker, each using 10 cores (threads) and 8 GB of memory
    processes=2,
    memory="16GiB",
    walltime="0-30:30",# walltime="0-00:30",
    log_directory="../dask/logs",  # folder for SLURM logs for each worker
    local_directory="../dask",  # folder for workers data
)

/opt/nesi/CS400_centos7_bdw/Python/3.9.5-gimkl-2020a/lib/python3.9/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41809 instead
  warnings.warn(


In [27]:
#cluster.adapt(minimum_jobs=20, maximum_jobs=200)
cluster.adapt(minimum_jobs=10, maximum_jobs=200)
client = Client(cluster)
client

Client Scheduler: tcp://192.168.94.156:34102 Dashboard: http://192.168.94.156:41809/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


## Flatten

In [28]:
#does not need to flatten
#bg = bg.flatten()

## Convert Bags to Dataframes

In [29]:
#convert from bags to dataframe
#ddf = bg.to_dataframe(meta={'name':'str', 'line':'int', 'severity':'str', 'message':'str', 'source':'str'})
ddf = bg.to_dataframe()

In [30]:
ddf

,beginline,endline,begincolumn,endcolumn,rule,ruleset,class,externalInfoUrl,priority,msg,method
npartitions=577,,,,,,,,,,,
,object,object,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...


In [31]:
ddf.columns

Index(['beginline', 'endline', 'begincolumn', 'endcolumn', 'rule', 'ruleset',
       'class', 'externalInfoUrl', 'priority', 'msg', 'method'],
      dtype='object')

In [32]:
ddf.head()

,beginline,endline,begincolumn,endcolumn,rule,ruleset,class,externalInfoUrl,priority,msg,method
0,2,2,1,21,UnnecessaryImport,Code Style,Code_1001005_1000723_311_0,https://pmd.github.io/pmd-6.38.0/pmd_rules_jav...,4,\nUnused import 'com.sun.jna.*'\n,
1,6,22,10,3,UnnecessaryModifier,Code Style,Code_1001005_1000723_311_0$Code,https://pmd.github.io/pmd-6.38.0/pmd_rules_jav...,3,\nUnnecessary modifier 'static' on interface '...,
2,7,8,19,49,UnnecessaryModifier,Code Style,Code_1001005_1000723_311_0$Code,https://pmd.github.io/pmd-6.38.0/pmd_rules_jav...,3,\nUnnecessary modifiers 'public static' on fie...,
3,10,10,19,56,UnnecessaryModifier,Code Style,Code_1001005_1000723_311_0$Code,https://pmd.github.io/pmd-6.38.0/pmd_rules_jav...,3,\nUnnecessary modifiers 'public static' on fie...,
4,11,11,19,58,UnnecessaryModifier,Code Style,Code_1001005_1000723_311_0$Code,https://pmd.github.io/pmd-6.38.0/pmd_rules_jav...,3,\nUnnecessary modifiers 'public static' on fie...,


In [33]:
ddf.tail()

,beginline,endline,begincolumn,endcolumn,rule,ruleset,class,externalInfoUrl,priority,msg,method
618,3,3,17,28,UnusedPrivateField,Best Practices,Code_9677845_9606054_320_1,https://pmd.github.io/pmd-6.38.0/pmd_rules_jav...,3,\nAvoid unused private fields such as 'superbi...,
619,5,5,1,20,UnnecessaryImport,Code Style,Code_9721129_9719864_349_1,https://pmd.github.io/pmd-6.38.0/pmd_rules_jav...,4,\nUnused import 'java.io.File'\n,
620,6,6,1,27,UnnecessaryImport,Code Style,Code_9721129_9719864_349_1,https://pmd.github.io/pmd-6.38.0/pmd_rules_jav...,4,\nUnused import 'java.io.IOException'\n,
621,10,10,17,17,UnusedLocalVariable,Best Practices,Code_9889597_9817516_187_9,https://pmd.github.io/pmd-6.38.0/pmd_rules_jav...,3,\nAvoid unused local variables such as 'o'.\n,collidedWith
622,2,2,12,21,OverrideBothEqualsAndHashcode,Error Prone,Code_9945526_9945329_420_8,https://pmd.github.io/pmd-6.38.0/pmd_rules_jav...,3,\nEnsure you override both equals() and hashCo...,hashCode


In [34]:
ddf.shape[0].compute()

221495